In [ ]:
import os

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import VotingClassifier

In [ ]:
train_data = pd.read_csv(os.path.join("data", "train_essays.csv"))
prompt_data = pd.read_csv(os.path.join("data", "train_prompts.csv"))
train_data.head()

In [ ]:
vectorizer = TfidfVectorizer()
vectorized_train_data = vectorizer.fit_transform(train_data["text"])
vectorized_train_data

In [ ]:
sgd_model1 = SGDClassifier(max_iter=5000, tol=1e-3, loss="modified_huber") 
sgd_model2 = SGDClassifier(max_iter=5000, tol=1e-3, loss="modified_huber", class_weight="balanced")
sgd_model3 = SGDClassifier(max_iter=10000, tol=5e-4, loss="modified_huber", early_stopping=True)

In [ ]:
model = VotingClassifier(estimators=[("sgd1", sgd_model1), ("sgd2", sgd_model2),("sgd3", sgd_model3)], voting="soft")

In [ ]:
model.fit(vectorized_train_data, train_data["generated"])
predictions = model.predict(vectorized_train_data)
submission = pd.DataFrame({"id": train_data["id"], "generated": predictions})
submission.head()